In [2]:
# importar pacotes necessários
import numpy as np
import pandas as pd

In [3]:
# definir parâmetros extras
pd.set_option('precision', 4)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [4]:
# importar pacotes usados na seleção do modelo e na medição da precisão
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

# importar os pacotes necessários para os algoritmos de regressão
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lars
from sklearn.linear_model import LassoLars
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import ARDRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import TheilSenRegressor
from sklearn.linear_model import HuberRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import RadiusNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.svm import NuSVR
from sklearn.svm import LinearSVR
from sklearn.tree import DecisionTreeRegressor

from xgboost import XGBRegressor

In [6]:
input_prefix = ''
#input_prefix = 'https://github.com/hjort/ai-labs/raw/master/jupyter/sales-russia/'

bzfile = '.bz2' # ''

## Training

In [7]:
df_train = pd.read_csv(input_prefix + 'train-data.csv' + bzfile, nrows=1000,
                       index_col='id')
print('shape:', df_train.shape)
df_train.head()

shape: (1000, 11)


,shop_id,item_id,date_block_num,item_cnt_month,city_code,item_category_id,subject_code,group_code,subgroup_code,year,month
id,,,,,,,,,,,
0,5,5037,0,0.0,5,19,793,6,13,2013,1
1,5,5037,1,0.0,5,19,793,6,13,2013,2
2,5,5037,2,0.0,5,19,793,6,13,2013,3
3,5,5037,3,0.0,5,19,793,6,13,2013,4
4,5,5037,4,0.0,5,19,793,6,13,2013,5


In [45]:
#for col in ('shop_id', 'item_id'):
#    df_train[col] = df_train[col].astype('category')

In [8]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 11 columns):
shop_id             1000 non-null int64
item_id             1000 non-null int64
date_block_num      1000 non-null int64
item_cnt_month      1000 non-null float64
city_code           1000 non-null int64
item_category_id    1000 non-null int64
subject_code        1000 non-null int64
group_code          1000 non-null int64
subgroup_code       1000 non-null int64
year                1000 non-null int64
month               1000 non-null int64
dtypes: float64(1), int64(10)
memory usage: 93.8 KB


In [47]:
df_train.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.9358e+06,2.9358e+06,2.9358e+06,2.9358e+06,2.9358e+06
mean,1.4570e+01,3.3002e+01,1.0197e+04,8.9085e+02,1.2426e+00
std,9.4230e+00,1.6227e+01,6.3243e+03,1.7298e+03,2.6188e+00
min,0.0000e+00,0.0000e+00,0.0000e+00,-1.0000e+00,-2.2000e+01
25%,7.0000e+00,2.2000e+01,4.4760e+03,2.4900e+02,1.0000e+00
50%,1.4000e+01,3.1000e+01,9.3430e+03,3.9900e+02,1.0000e+00
75%,2.3000e+01,4.7000e+01,1.5684e+04,9.9900e+02,1.0000e+00
max,3.3000e+01,5.9000e+01,2.2169e+04,3.0798e+05,2.1690e+03


## Testing

In [9]:
df_test = pd.read_csv(
    input_prefix + 'test-data.csv' + bzfile, nrows=1000,
    index_col='id')
print('shape:', df_test.shape)
df_test.head()

shape: (1000, 10)


,shop_id,item_id,date_block_num,city_code,item_category_id,subject_code,group_code,subgroup_code,year,month
id,,,,,,,,,,
0,5,5037,34,5,19,793,6,13,2015,11
1,5,5320,34,5,55,818,13,6,2015,11
2,5,5233,34,5,19,786,6,13,2015,11
3,5,5232,34,5,23,786,6,19,2015,11
4,5,5268,34,5,20,786,6,14,2015,11


In [39]:
#for col in ['shop_id', 'item_id']:
#    df_test[col] = df_test[col].astype('category')

In [10]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 10 columns):
shop_id             1000 non-null int64
item_id             1000 non-null int64
date_block_num      1000 non-null int64
city_code           1000 non-null int64
item_category_id    1000 non-null int64
subject_code        1000 non-null int64
group_code          1000 non-null int64
subgroup_code       1000 non-null int64
year                1000 non-null int64
month               1000 non-null int64
dtypes: int64(10)
memory usage: 85.9 KB


In [11]:
df_test.describe()

,shop_id,item_id,date_block_num,city_code,item_category_id,subject_code,group_code,subgroup_code,year,month
count,1000.0,1000.0000,1000.0,1000.0,1000.0000,1000.0000,1000.0000,1000.0000,1000.0,1000.0
mean,5.0,9897.6070,34.0,5.0,41.4970,1874.3100,10.3640,21.7160,2015.0,11.0
std,0.0,6514.7495,0.0,0.0,17.8167,1433.4781,3.2873,16.0978,0.0,0.0
min,5.0,31.0000,34.0,5.0,2.0000,1.0000,2.0000,1.0000,2015.0,11.0
25%,5.0,3731.7500,34.0,5.0,24.0000,524.0000,6.0000,8.0000,2015.0,11.0
50%,5.0,9102.0000,34.0,5.0,40.0000,1550.0000,11.0000,15.0000,2015.0,11.0
75%,5.0,15340.5000,34.0,5.0,55.0000,3065.0000,13.0000,35.0000,2015.0,11.0
max,5.0,22167.0000,34.0,5.0,83.0000,4655.0000,18.0000,55.0000,2015.0,11.0


In [12]:
data = df_train

In [13]:
# definir dados de entrada
X = data.drop(['item_cnt_month'], axis=1) # tudo, exceto a coluna alvo
y = data['item_cnt_month'] # apenas a coluna alvo

print('Forma dos dados originais:', X.shape, y.shape)

Forma dos dados originais: (1000, 10) (1000,)


## Treinamento dos modelos preditivos

In [14]:
MSE = 'neg_mean_squared_error'

models = {}

parallel_jobs = 2#8 # paralelização

In [15]:
from datetime import datetime

# avalia o desempenho do modelo, retornando o valor do MSE
def evaluate_model_cv(name, model, X=X, y=y):
    start = datetime.now()
    kfold = KFold(n_splits=10, random_state=42)
    results = cross_val_score(model, X, y, cv=kfold, scoring=MSE, verbose=1, n_jobs=parallel_jobs)
    end = datetime.now()
    elapsed = int((end - start).total_seconds() * 1000)
    score = (-1) * results.mean()
    stddev = results.std()
    models[name] = {'model': model, 'score': score, 'stddev': stddev, 'elapsed': elapsed}
    print(model, '\nScore: %.2f (+/- %.2f) [%5s ms]' % (score, stddev, elapsed))
    return score, stddev, elapsed

In [16]:
# faz o ajuste fino do modelo, calculando os melhores hiperparâmetros
def fine_tune_model(model, params, X=X, y=y):
  print('\nFine Tuning Model:')
  print(model, "\nparams:", params)
  kfold = KFold(n_splits=10, random_state=42)
  grid = GridSearchCV(estimator=model, param_grid=params, scoring=MSE, cv=kfold, verbose=1, n_jobs=parallel_jobs)
  grid.fit(X, y)
  print('\nGrid Best Score: %.2f' % (grid.best_score_ * (-1)))
  print('Best Params:', grid.best_params_)
  return grid

### Avaliação e ajuste fino de cada modelo preditivo

- https://scikit-learn.org/stable/modules/classes.html

In [ ]:
model = KNeighborsRegressor(n_jobs=-1, n_neighbors=11, weights='distance')
%time evaluate_model_cv('KNN', model)

#n_neighbors=5, weights=’uniform’, algorithm=’auto’, leaf_size=30, p=2, metric=’minkowski’,
#metric_params=None, n_jobs=None

params = {
    'n_neighbors': [1, 3, 5, 7, 9, 11, 13],
    'weights': ['uniform', 'distance']
}
#fine_tune_model(model, params)

In [17]:
model = RandomForestRegressor(random_state=42, n_jobs=-1, n_estimators=100, max_depth=3)
%time evaluate_model_cv('RF', model)

#n_estimators=’warn’, criterion=’mse’, max_depth=None, min_samples_split=2, min_samples_leaf=1, 
#min_weight_fraction_leaf=0.0, max_features=’auto’, max_leaf_nodes=None, min_impurity_decrease=0.0, 
#min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, 
#verbose=0, warm_start=False

params = {
    'n_estimators': [5, 10, 25, 50, 75, 100],
    'max_depth': [None, 3, 5, 7, 9, 11, 13]
}
#fine_tune_model(model, params)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=3,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                      oob_score=False, random_state=42, verbose=0,
                      warm_start=False) 
Score: 3.24 (+/- 2.49) [17371 ms]
CPU times: user 292 ms, sys: 84 ms, total: 376 ms
Wall time: 17.4 s


[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:   17.1s finished


In [18]:
model = GradientBoostingRegressor(random_state=42,
    learning_rate=0.05, n_estimators=100, max_depth=4, max_features=0.85)
%time evaluate_model_cv('GB', model)

#loss=’ls’, learning_rate=0.1, n_estimators=100, subsample=1.0, criterion=’friedman_mse’, min_samples_split=2,
#min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, 
#min_impurity_split=None, init=None, random_state=None, max_features=None, alpha=0.9, verbose=0, 
#max_leaf_nodes=None, warm_start=False, presort=’auto’, validation_fraction=0.1, n_iter_no_change=None, 
#tol=0.0001

params = dict(
    n_estimators=[100, 250, 500],
    max_features=[0.75, 0.85, 1.0],
    max_depth=[4, 8, 12],
    learning_rate=[0.05, 0.1, 0.15],
    #subsample=[0.4, 0.6, 0.8]
)
#fine_tune_model(model, params)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.05, loss='ls', max_depth=4,
                          max_features=0.85, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=42, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False) 
Score: 3.71 (+/- 2.46) [ 3564 ms]
CPU times: user 220 ms, sys: 16 ms, total: 236 ms
Wall time: 3.57 s


[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    3.6s finished


In [19]:
model = ExtraTreesRegressor(random_state=42, n_jobs=-1, n_estimators=75, max_features=0.85)
%time evaluate_model_cv('ET', model)

#n_estimators=’warn’, criterion=’mse’, max_depth=None, min_samples_split=2, min_samples_leaf=1, 
#min_weight_fraction_leaf=0.0, max_features=’auto’, max_leaf_nodes=None, min_impurity_decrease=0.0, 
#min_impurity_split=None, bootstrap=False, oob_score=False, n_jobs=None, random_state=None, verbose=0,
#warm_start=False

params = dict(
    n_estimators=[50, 75, 100, 200],
    max_features=['auto', 0.75, 0.85, 1.0]
)
#fine_tune_model(model, params)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
                    max_features=0.85, max_leaf_nodes=None,
                    min_impurity_decrease=0.0, min_impurity_split=None,
                    min_samples_leaf=1, min_samples_split=2,
                    min_weight_fraction_leaf=0.0, n_estimators=75, n_jobs=-1,
                    oob_score=False, random_state=42, verbose=0,
                    warm_start=False) 
Score: 4.25 (+/- 3.06) [ 6193 ms]
CPU times: user 236 ms, sys: 8 ms, total: 244 ms
Wall time: 6.2 s


[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    6.2s finished


In [20]:
model = BaggingRegressor(random_state=42, n_jobs=-1, base_estimator=DecisionTreeRegressor(), max_features=0.5, n_estimators=200)
%time evaluate_model_cv('BG', model)

#base_estimator=None, n_estimators=10, max_samples=1.0, max_features=1.0, bootstrap=True, 
#bootstrap_features=False, oob_score=False, warm_start=False, n_jobs=None, random_state=None, verbose=0

params = dict(
    n_estimators=[50, 75, 100, 200],
    max_features=[0.5, 0.75, 1.0]
)
#fine_tune_model(model, params)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


BaggingRegressor(base_estimator=DecisionTreeRegressor(criterion='mse',
                                                      max_depth=None,
                                                      max_features=None,
                                                      max_leaf_nodes=None,
                                                      min_impurity_decrease=0.0,
                                                      min_impurity_split=None,
                                                      min_samples_leaf=1,
                                                      min_samples_split=2,
                                                      min_weight_fraction_leaf=0.0,
                                                      presort=False,
                                                      random_state=None,
                                                      splitter='best'),
                 bootstrap=True, bootstrap_features=False, max_features=0.5,
                 max_sample

[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:   13.6s finished


In [21]:
model = AdaBoostRegressor(random_state=42, n_estimators=100, base_estimator=DecisionTreeRegressor())
%time evaluate_model_cv('ABDT', model)

# base_estimator=None, n_estimators=50, learning_rate=1.0, algorithm=’SAMME.R’, random_state=None

params = dict(
    n_estimators=[50, 75, 100, 200]
)
#fine_tune_model(model, params)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mse',
                                                       max_depth=None,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                                       min_weight_fraction_leaf=0.0,
                                                       presort=False,
                                                       random_state=None,
                                                       splitter='best'),
                  learning_rate=1.0, loss='linear', n_estimators=100,
                  rand

[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:   11.1s finished


## Comparação final entre os algoritmos

In [22]:
results = []
names = []
scores = []
stddevs = []
times = []

best_model = None
highest_score = None

for name, model in models.items():
    score, stddev, elapsed = model['score'], model['stddev'], model['elapsed']
    results.append((score, stddev))
    names.append(name)
    scores.append(score)
    stddevs.append(stddev)
    times.append(elapsed)
    
    if not highest_score or score < highest_score:
        best_model = model['model']
        highest_score = score

In [23]:
results_df = pd.DataFrame({'Model Name': names, 'Score': scores, 'Std Dev': stddevs, 'Time (ms)': times})
results_df.sort_values(by=['Score', 'Time (ms)'], ascending=[True, False])

,Model Name,Score,Std Dev,Time (ms)
3,BG,3.0840,2.3475,13583
0,RF,3.2449,2.4854,17371
1,GB,3.7061,2.4617,3564
2,ET,4.2489,3.0644,6193
4,ABDT,4.6450,3.1968,11118


In [24]:
print('Best Model:\n\n%s' % (best_model))

Best Model:

BaggingRegressor(base_estimator=DecisionTreeRegressor(criterion='mse',
                                                      max_depth=None,
                                                      max_features=None,
                                                      max_leaf_nodes=None,
                                                      min_impurity_decrease=0.0,
                                                      min_impurity_split=None,
                                                      min_samples_leaf=1,
                                                      min_samples_split=2,
                                                      min_weight_fraction_leaf=0.0,
                                                      presort=False,
                                                      random_state=None,
                                                      splitter='best'),
                 bootstrap=True, bootstrap_features=False, max_features=0.5,
              

In [25]:
model = best_model
model.fit(X, y)

BaggingRegressor(base_estimator=DecisionTreeRegressor(criterion='mse',
                                                      max_depth=None,
                                                      max_features=None,
                                                      max_leaf_nodes=None,
                                                      min_impurity_decrease=0.0,
                                                      min_impurity_split=None,
                                                      min_samples_leaf=1,
                                                      min_samples_split=2,
                                                      min_weight_fraction_leaf=0.0,
                                                      presort=False,
                                                      random_state=None,
                                                      splitter='best'),
                 bootstrap=True, bootstrap_features=False, max_features=0.5,
                 max_sample

In [26]:
X.head()

,shop_id,item_id,date_block_num,city_code,item_category_id,subject_code,group_code,subgroup_code,year,month
id,,,,,,,,,,
0,5,5037,0,5,19,793,6,13,2013,1
1,5,5037,1,5,19,793,6,13,2013,2
2,5,5037,2,5,19,793,6,13,2013,3
3,5,5037,3,5,19,793,6,13,2013,4
4,5,5037,4,5,19,793,6,13,2013,5


In [27]:
del(X)
del(y)

## ....

In [28]:
# colocar as colunas na mesma ordem dos dados de treino
cols = df_train.columns.drop(['item_cnt_month'])
cols

Index(['shop_id', 'item_id', 'date_block_num', 'city_code', 'item_category_id',
       'subject_code', 'group_code', 'subgroup_code', 'year', 'month'],
      dtype='object')

In [29]:
# selecionar atributos para o modelo
X_pred = df_test[cols]

In [30]:
# realizar a previsão
y_pred = best_model.predict(X_pred)

In [33]:
df_final = pd.DataFrame(data=y_pred, columns=['item_cnt_month'], index=df_test.index)
df_final.index.names = ['ID']
df_final.head()

,item_cnt_month
ID,
0,0.6867
1,0.4094
2,0.8301
3,0.3656
4,0.1679


In [34]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 1 columns):
item_cnt_month    1000 non-null float64
dtypes: float64(1)
memory usage: 15.6 KB


In [35]:
!test -d submissions/ || mkdir submissions/

In [36]:
df_final.to_csv('submissions/submission.csv')